In [7]:
import torch

In [8]:
torch.cuda.is_available()

True

In [1]:
import torch
from transformers import AutoTokenizer, TextStreamer, GenerationConfig
from attention_sinks import AutoModelForCausalLM

device = "cuda"
# model_id = "meta-llama/Llama-2-7b-hf"
# model_id = "mistralai/Mistral-7B-v0.1"
# model_id = "mosaicml/mpt-7b"
# model_id = "tiiuae/falcon-7b"
# model_id = "EleutherAI/pythia-6.9b-deduped"
# Note: instruct or chat models also work.
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# Load the chosen model and corresponding tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # for efficiency:
    device_map=device,
    # load_in_4bit=True
    torch_dtype=torch.float16,
    # `attention_sinks`-specific arguments:
    attention_sink_size=4,
    attention_sink_window_size=252, # <- Low for the sak of faster generation
)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

# Our input text
text = "Generate 100 realistic and plausible short spam messages including fake phone number, addr or price. examples : Congratulations! You've won a $1,000 prize in our special event. To claim your prize, click this link now: www.prizelink.com/claim or call +125123520., \
    Your package is pending delivery. To complete delivery, pay the shipping fee via this link: www.parceltrack.com/pay. For queries, contact +1024182314.,\
         Find your soulmate today! Sign up at www.loveconnection.net to start your love story.,\
         don’t miss out on our exclusive content! Your subscription ends soon. Renew now at www.renewsubscription.com.,\
         Immediate hiring! Apply now for an exclusive job opportunity with us. Visit www.careeropportunity.net for more information.\
        rules : 1. don't add hashtag 2.only generate spam SMS text message 3.message must include phone number, addr or price or links."


prompt = "Generate 100 fake spam SMS text message for text data augmentation. for EXAMPLE : \
[Prize Notification Spam]\
Congratulations! You've won a $1,000 prize in our special event. To claim your prize, click this link now: www.sampleprizelink.com/claim or call +13526450.\
You're our lucky $500 gift card winner! To get your gift, visit www.winnernotice.net immediately. Promo ends today!\
[Parcel/Delivery Spam]\
Your package is pending delivery. To complete delivery, pay the shipping fee via this link: www.fakeparceltrack.com/pay. For queries, contact +12345316378.\
We couldn't deliver your parcel today. Reschedule now at www.misseddelivery.com or it will be returned to sender.\
[Banking/Financial Fraud Spam]\
Alert: Your Bank XYZ account has a security issue. Secure your account by visiting www.bankxyz-security.com. Call us at +113267892 for support.\
Important: We've noticed unusual activity in your account. Confirm your identity at www.fakebankalert.com to avoid suspension.\
[Fake Alert Messages]\
Critical Warning: Your software license has expired. Renew immediately by clicking here: www.softwarelicenseupdate.com.\
Security Alert: Your computer has been infected with a virus. Click here for a free cleanup: www.computercleanup.com.\
[Fake Employment/Job Offer Spam]\
You are the perfect candidate for our new position! Send your CV now and book your interview slot at www.joboffer.com.\
Immediate hiring! Apply now for an exclusive job opportunity with us. Visit www.careeropportunity.net for more information.\
[Insurance Advertisement Spam]\
Protect your family with affordable insurance rates! Check out the best offer now by clicking www.insurecheap.info.\
Life insurance plans at a low cost just for you! Explore more at www.lifeinsurepromo.com.\
[Fake Surveys/Polls]\
Invest just a few minutes to complete this survey and win a $50 voucher! Participate now at www.surveyreward.com.\
Your opinion matters! Fill out our quick survey and you could win $100 in cash! Click here: www.surveyprize.net.\
[Romance/Dating Scams]\
I want to meet you! Check out my profile here and let's chat: www.fakeromance.com.\
Find your soulmate today! Sign up at www.loveconnection.net to start your love story.\
[Healthcare/Product Sales Spam]\
Rediscover your health with our innovative wellness supplement, now at a discount! Visit www.healthboostnow.com.\
Exclusive offer on our new health product! Improve your life quality today: www.newhealthproduct.com.\
[Subscription/Service Cancellation Spam]\
Your subscription is about to expire. Continue enjoying our service by renewing here: www.subscribenow.com.\
Don’t miss out on our exclusive content! Your subscription ends soon. Renew now at www.renewsubscription.com.\
RULES : 1. don't add hashtag 2.only generate spam SMS text message 3.message must include phone number, addr or price or links."
# Encode the text
output_file_path = "generated_spam.txt"
input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)
with open(output_file_path, "a", encoding="utf-8") as output_file:
    with torch.no_grad():
        # A TextStreamer prints tokens as they're being generated
        streamer = TextStreamer(tokenizer)
        generated_tokens = model.generate(
            input_ids,
            generation_config=GenerationConfig(
                # use_cache=True is required, the rest can be changed up.
                use_cache=True,
                min_new_tokens=100_000,
                max_new_tokens=1_000_000,
                penalty_alpha=0.6,
                top_k=3,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            ),
            streamer=streamer,
        )
        # Decode the final generated text

        output_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
        output_file.write(output_text)

/home/tbelldev/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.25s/it]
[Attention Sinks] Injected Position Shifting into 32 attention classes.
[Attention Sinks] Injected Attention Sink KV Cache into 1 model class.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<s> Generate 100 realistic and plausible short spam messages including fake phone number, addr or price. examples : Congratulations! You've won a $1,000 prize in our special event. To claim your prize, click this link now: www.prizelink.com/claim or call +125123520.,     Your package is pending delivery. To complete delivery, pay the shipping fee via this link: www.parceltrack.com/pay. For queries, contact +1024182314.,         Find your soulmate today! Sign up at www.loveconnection.net to start your love story.,         don’t miss out on our exclusive content! Your subscription ends soon. Renew now at www.renewsubscription.com.,         Immediate hiring! Apply now for an exclusive job opportunity with us. Visit www.careeropportunity.net for more information.        rules : 1. don't add hashtag 2.only generate spam SMS text message 3.message must include phone number, addr or price or links.

1. Your order is ready to be shipped! Click this link to track your package: www.shippingtrack

KeyboardInterrupt: 